---
# Unit07: The Network and Array Methods

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)

#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook

---
## Read a data set from TTB22
As ondas sísmicas geradas pela transferência da energia do fluxo turbulento no estuário do Rio Amazonas, resultante das suas grandes marés, por forças friccionais às rugosidades do seu fundo. O arranjo sísmico foi instalado na Ilha de Tatuoca, como tal o arranjo está circundado por uma distribuição de fontes no leito do rio. Fontes e geofones estão na superfície, tornando as ondas de superfície francamente dominantes em relação às de volume.
### The Experiment
- **The Receiver Array**
The array has 24 GS-20DX vertical geophones hooked to a $L=69$m cable, using takeouts spaced $\delta l=6$m from each other. The GS-20DX geophones have a natural frequency of $f_n=10\textrm{Hz}$, and a spurious frequency $f_{sp}>250$Hz. The array has a irregular circular shape, deployed in the Southern tip of the island, with its center at $\left(1^{\circ}12^{\prime}6.93^{\prime\prime}\textrm{S},48^{\circ}30^{\prime}23.39^{\prime\prime}\textrm{S}\right)$.

<img src="./ttb22.png" width="600">

- **The data**
Each of the 12 traces of **file 3804** is $\Delta T=60$s long, with a sampling frequency of $f_{s}=250$Hz. This file was recorded on 2022-04-02, begining at 13h 56min 41s. That was during the maximum gradient of the local ebb tide.

<img src="./tide.png" width="500">

In [ ]:
"""
====================== A local routine to read a CSV file ======================
Read a cvs file and stores the information in an object numpy array
"""
def RGloc(filename):
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            data = []
            for line in lines[1:]:               # Skip header row
                parts = line.strip().split(',')
                data.append([int(parts[0]), float(parts[1]), float(parts[2])])
        return np.array(data, dtype=object)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None                                 
#
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
"""
#
#------ Read the seismic data
filename = '3804'
print(f">> Read with data file {filename}")
filename = '../Data/'+filename+'.dat'   
#------- Read the data file as a SEG2 object
st = read(filename)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f"1) Gather acquired on {st[0].stats.starttime}, has {len(st)} geophones along {dummy}m.")
dummy = (UTCDateTime(st[0].stats.endtime) - UTCDateTime(st[0].stats.starttime))
print(f"2) Each {dummy}s-long trace has {int(st[0].stats.npts)} data points.")
print(f"3) The sampling frequency is {st[0].stats.sampling_rate}Hz")
#
"""
====================== READ THE PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = RGloc('../Data/'+'ttb_loc.dat')
#
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

---
## Have a look at one trace.

In [ ]:
#------ Pick up a phone
dummy = np.random.randint(1, len(st)+1)
ent = input(f' Enter a phone [dflt=random] ')
ent = int( ent.rstrip().split(' ')[0] ) if ent else dummy
#------- The trace
trZ = st[ent - 1].copy()
#       +───────+─> Trace = Phone -1 
#------- Deep copy of the trace
tr0 = trZ.copy()
#------ Relative time: nummpy array
time = trZ.times(type="relative")
#------- Filter the trace
#--- Filter parameters
MTparam = [1, 1, 'bp',  5., 50., 1, 0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
trZ = u.otrstr(trZ, MTparam)
#------ Plot Spectrogram
p.Pspect(time, trZ)
#
#-------- Save trace for the next cell + append to the processing flux
tr0, trZ, _ = u.AuxReset(tr0, trZ)

---
## Detect events with the trigger algorithms
- The STA window length $\Delta T_{STA}$ measures the average amplitude of the seismic **signals**.
- The LTA window length $\Delta T_{LTA}$, measures the average amplitude of the seismic **noise**, with lengths **larger** than a few periods of noise fluctuations.
- The relative occurences of **positive**, **false positive** and **missed positive** is conditioned by the STA/LTA trigger threshold level $\mathcal{T}_{STA/LTA}$.
- The termination of data recording is estimated with the STA/LTA detrigger threshold level $\widetilde{\mathcal{T}}_{STA/LTA}$.

$\mathcal{A}\left(s+n\right)\rightarrow0$ | $\mathcal{A}\left(s+n\right)\rightarrow\infty$ |$\qquad\mathcal{T}_{STA/LTA}\rightarrow0$ | $\qquad\mathcal{T}_{STA/LTA}\rightarrow\infty$ |
| :- | :- | :- | :- |
| $\mathcal{T}_{STA/LTA}\approx4$ | $\mathcal{T}_{STA/LTA}\gtrsim8$ | $\textrm{events}+\textrm{false-trigg.}\rightarrow\infty$ | $\textrm{misses}\rightarrow\infty,\,\textrm{false-trigg.}\rightarrow 0$ |
| $\widetilde{\mathcal{T}}_{STA/LTA}\approx2-3$ | $\widetilde{\mathcal{T}}_{STA/LTA}>3$ | stationary seismic noise | irregular seismic noise levels|

<img src="StaLta2.png" width="500">

### STA/LTA recursive detector
- Work with a single trace.
- Try different STA and LTA lengths. Rule of thumb $\rightarrow$ is that $\Delta T_{STA}\approx\textrm{signal}$ **and** $\Delta T_{LTA}\gg\lambda_{\textrm{low freq. noise}}$

In [ ]:
#------ Import Packages
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import recursive_sta_lta
#
print(f">> Trigger packages imported.")

"""
Parameters:
nsta(int) – Length of short time average window in samples
nlta(int) – Length of long time average window in samples
"""
print(f">> Sampling interval = {np.round(trZ.stats.delta, 5)}s")
#
#------ Enter STA and LTA in s.
ent = input(f'<< Enter STA and LTA in s: ')
ent = ent.rstrip().split(' ')
sta = float(ent[0])
lta = float(ent[1])
#
dummy = sta
sta = int(sta * trZ.stats.sampling_rate)
print(f">> sta = {dummy}s or {sta} data points")
#
dummy = lta
lta = int(lta * trZ.stats.sampling_rate)
print(f">> lta = {dummy}s or {lta} data points")
#
#------ Enter trigger and detrigger thresholds.
ent = input(f'<< Enter trigger and detrigger thresholds: ')
ent = ent.rstrip().split(' ')
trg  = float(ent[0])
dtrg = float(ent[1])
#
#------ Hints: STA, LTA = 0.5 5;  trigger, detrigger thresholds = 2 1

### Apply Recursive STA/LTA

In [ ]:
#------ STA and LTA hints: 5 10
#------ Trigger and detrigger hints: 1.5 0.5
cft = recursive_sta_lta(trZ.data, sta, lta)
#
plot_trigger(trZ, cft, trg, dtrg)

---
### Matched-filter detection
The seismogram suggests river generated signals have similar waveforms, a similarity that could be used for detection. Based on this we can use the correlation between a **template waveform** and the data to generate a characteristic function, and with it, detect events. The pros of correlation detectors are:
- They can potentially detect lower S/N events;
- They do not require amplitudes to return to low levels, so close in time multiple events are more likely to be detected.

Conversely their cons are that they need a **template**, implying a restriction to events close in space and with similar focal mechanisms.

A tutorial in ObsPy can be found at [here](https://docs.obspy.org/master/tutorial/code_snippets/xcorr_detector.html), and a package on the subject is at [EQcorrscan](https://github.com/eqcorrscan/EQcorrscan).

In [ ]:
#------ Plot Trace+Spectrogram
p.Pspect(time, trZ)

In [ ]:
"""
Create a catalog of templates. 
"""
cat = []
while True:
  trTplt = trZ.copy()
  ent = input(f'<< Enter pck0 and pck1 in s (rtn=exit): ')
  if not ent: break
  ent = ent.rstrip().split(' ')
  pck0 = trTplt.stats.starttime + float(ent[0])
  pck1 = trTplt.stats.starttime + float(ent[1])
#------ Trim the template
  trTplt.trim(pck0, pck1)
  print(f">> Template is at [{pck0}, {pck1}] with {len(trTplt)} data points.")
#------ Relative time: nummpy array
  dummy = trTplt.times(type="relative")
#------ fft
  fNy = 0.8 * trTplt.stats.sampling_rate / 2.
  FtrTplt = np.fft.rfft(trTplt.data)
  freq = np.linspace(0, fNy, len(FtrTplt))
  p.pltTrSp( dummy, trTplt.data, freq, abs(FtrTplt),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='Hz', y2label='Spec.', y2log=True, clr2 = 'r' )
#------ Construct catalog
  cat.append(trTplt)   
#------ The catalog
trTplt = cat
print(f">> Template catalog has {len(trTplt)} templates.")

---
## Detect events with the Matched-filter detection algorithms
Work with a single trace. It is necessary to provide two paramters:
- The correlation threshold within the normalised correlation range [-1, 1]
- The triger off time in s. This is the minimum distance between events.

In [ ]:
#------ Import Packages
from obspy.signal.cross_correlation import correlation_detector
import pprint
print(f">> Trigger packages imported.")
#
#------ Enter height and distance. Hint: .6 2
ent = input(f'<< Enter correlation threshold and triger off time(s): ')
ent = ent.rstrip().split(' ')
corthr   = float(ent[0])
trgoff   = float(ent[1])
#
#------ Produce ObsPy stream
stream  = Stream([trZ])
#tmplstr = [Stream([template]) for template in trTplt]
#tmplstr = Stream([trTplt])
#
#------ Detections. Free memory on each loop.
detect = []
for template in trTplt:
    tmplstr = Stream([template])
    dets, sims = correlation_detector(
                   stream=stream, templates=[tmplstr],
                   heights=corthr, distance=trgoff, plot=stream)
    detect.extend(dets)
#
print("Made {0} detections from {1} templates".format(
    len(detect), len(trTplt)))